<a href="https://colab.research.google.com/github/joaochenriques/GeneratorCalib/blob/main/GenFilterPaper_PP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
CASES = {
    0: ( 'T00', '17VPMsByFk7y7hvfCulCVsXU9NFCYm80m' ),
    1: ( 'T01', '1ntRq9wLbatJs7RiALXIoL55ou_dHjGpm' ),
}

CASE = CASES[1]
CASE

('T01', '1ntRq9wLbatJs7RiALXIoL55ou_dHjGpm')

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import matplotlib.pyplot as mpl
import numpy as np
import scipy.io as sio
import h5py

In [4]:
!pip install dataclassy 
from dataclassy import dataclass

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
# If running python on Windows operating system, copy the file:
# https://raw.githubusercontent.com/joaochenriques/ipynb_libs/main/mpl_utils.py
# to the working folder before running the notebook

import pathlib
if not pathlib.Path("mpl_utils.py").exists():
  !curl -O https://raw.githubusercontent.com/joaochenriques/ipynb_libs/main/mpl_utils.py 

import mpl_utils as mut
mut.config_plots()

%config InlineBackend.figure_formats = ['svg']
mpl.rcParams["figure.figsize"] = (6,4.5)

In [6]:
!pip install gdown &> /dev/null
import gdown

In [7]:
def filters_sym_6th( pnts_seq ):

  b_lst = []
  for Npts in pnts_seq:
    b = np.zeros( Npts )

    float_Npts = float( Npts )
    hpts = Npts / 2

    if Npts % 2 == 0:
      print( ' Number of points should be an odd ', Npts )
      exit(1)

    CONST6_0 = (   1225.E0*float_Npts**6 -  57575.E0*float_Npts**4 +  \
                 605395.E0*float_Npts**2 - 952245.E0 ) / 64.E0
    CONST6_2 = ( -11025.E0*float_Npts**4 + 330750.E0*float_Npts**2
               -1507485.E0 ) / 16.E0
    CONST6_4 = (  24255.E0*float_Npts**2 - 347655.E0 ) / 4.E0
    CONST6_6 = -15015.E0
    Denomi_6 =  4.E0 * float_Npts * ( float_Npts**6 - 56.E0*float_Npts**4
            + 784.E0*float_Npts**2 - 2304.E0 )

    mid_Coef = np.array( [ CONST6_0 / Denomi_6 ] )
    pl = np.array( [CONST6_6, 0.0, CONST6_4, 0.0, CONST6_2, 0.0, CONST6_0] )

    float_I = np.arange( 1.0, hpts + 0.1, 1.0 )

    Coefs = np.polyval( pl, float_I ) / Denomi_6

    lst = np.concatenate( ( Coefs[::-1], mid_Coef, Coefs ) )
    b_lst.append( lst )
    
  return b_lst

In [8]:
def filter_multiply( filter, signal ):

  half_window = int( filter.size / 2 )

  if filter.size % 2 == 0:
    raise "ERROR@filter_multiply: filter size is even."

  out_signal = np.zeros( signal.size - 2*half_window )


  ej = filter.size;
  hw = int( ej / 2 )
  ei = int( signal.size - 2*hw )

  for i in range( 0, ei ):
      out_signal[i] = np.dot( signal[i:i+ej], filter )

  return out_signal

In [9]:
def sym_filters_czo_ends( b_lst, y ):
  # continuous zero orders ends
  window = 4*len( b_lst[0] )

  half_window = 0
  for b in b_lst:
    half_window += int( len(b) / 2 )

  firstvals = np.ones( half_window ) * np.average( y[ :window] )
  lastvals  = np.ones( half_window ) * np.average( y[-window:] )
  y = np.concatenate( ( firstvals, y, lastvals ) )

  for b in b_lst:
    y = filter_multiply( b, y )

  return y

In [10]:
def LS_fitting( time, signal, degree ):
  return np.poly1d( np.polyfit( time, signal, degree ) )

In [11]:
def sym_filters_cls_ends( b_lst, y, ends_degree = 6, mult = 1 ):
  # continuous least-square ends
  for b in b_lst:
    window      = mult*len(b)
    half_window = int( len(b) / 2 )

    time        = np.arange( 0, window+0.5, 1.0 )

    left_time   = time[ :half_window ]
    left_y      = y[ : window + 1 ]
    poly_left   = LS_fitting( time, left_y,  ends_degree )
    left_vals   = poly_left ( left_time )

    right_time  = time[ -half_window: ]
    right_y     = y[ -window-1: ]
    poly_right  = LS_fitting( time, right_y, ends_degree )
    right_vals  = poly_right( right_time )

    y = filter_multiply( b, y )
    y = np.concatenate( ( left_vals, y, right_vals ) )

  return y

In [12]:
def filter_var( HF_seq, LF_seq, time, signal ):

  initial_pnts = int( 62500*5+1 )
  discard_pnts = int( 62500*2.4 )

  time   = time[ -initial_pnts-1: -1 ]
  signal = signal[ -initial_pnts-1: -1 ]

  HF_filter = filters_sym_6th( HF_seq )
  LF_filter = filters_sym_6th( LF_seq )

  HF_signal  = sym_filters_cls_ends( HF_filter, signal, 6, 1 )
  LF_signal  = sym_filters_czo_ends( LF_filter, signal )
  BP_signal  = HF_signal.copy()
  BP_signal -= LF_signal

  return time[discard_pnts-1:-discard_pnts], \
         BP_signal[discard_pnts-1:-discard_pnts], \
         signal[discard_pnts-1:-discard_pnts]

In [16]:
TEST_CASE_NAME = CASE[0]
tests_filename = CASE[0]
gdown_filename = CASE[1]

dat_filename = TEST_CASE_NAME + '.mat'
dat_filename

'T01.mat'

In [14]:
if not pathlib.Path( dat_filename ).exists():
    gdown.download( id=gdown_filename, output=dat_filename, quiet=False )

Downloading...
From: https://drive.google.com/uc?id=1ntRq9wLbatJs7RiALXIoL55ou_dHjGpm
To: /content/T01.mat
100%|██████████| 2.11G/2.11G [00:46<00:00, 45.7MB/s]


In [ ]:
hf = h5py.File( dat_filename, 'r' )

In [15]:
mat_contents = sio.loadmat( mat_filename, squeeze_me = True, struct_as_record = False )
mat_contents['TData']._fieldnames

NotImplementedError: ignored

In [ ]:
Time = mat_contents['TData'].Time

In [ ]:
u_HF_seq = np.array( [1063, 1015,  967,  919,  871,  811] )
u_LF_seq = np.array( [17419, 16603, 15787, 14971, 14155, 13315] )

In [ ]:
def FilterSignal( coeffs, time, unf_signal ):

  NS = unf_signal.shape[0]
  NC = coeffs.shape[0]

  assert( NC % 2 > 0 ) # assert that is a centred filter (odd)

  flt_signal = np.convolve( unf_signal, coeffs, 'valid')
  hNC = int( NC / 2 )
  flt_time = time[ hNC: NS-hNC ]

  assert( flt_time.shape[0] == flt_signal.shape[0] )

  return flt_time, flt_signal

## filtering u1u2outGen

In [ ]:
u1u2out = mat_contents['TData'].u1u2out
LP_coeffs = filters_sym_6th( u_HF_seq )
flt_time, flt_u1u2out = FilterSignal( LP_coeffs[0], Time, u1u2out )

mpl.plot( Time, u1u2out, '-y' )
mpl.plot( flt_time, flt_u1u2out, '-b', lw=1.5 )

mpl.xlim( 0, 0.01 )
mpl.ylim( -7, 7 )

In [ ]:
( time, u1u2outGen, u1u2outGenOrg ) = \
      filter_var( u_HF_seq, u_LF_seq, glb_time, mat_contents['TData'].u1u2out )
u1u2outGen *= 80.0
u1u2outGenOrg *= 80.0

mpl.plot( time, u1u2outGenOrg, '-y' )
mpl.plot( time, u1u2outGen, '-b', lw=1.5 )
mpl.xlabel( 'time' )
mpl.ylabel( 'u1u2outGen' )